In [1]:
import pandas as pd

from dask import compute, delayed
from dask.diagnostics import ProgressBar
ProgressBar().register()

In [46]:
# from xlsx2csv import Xlsx2csv
# from io import StringIO
# import pandas as pd


# def read_excel(path: str, sheet_name: str=None) -> pd.DataFrame:
#     buffer = StringIO()
#     Xlsx2csv(path, outputencoding="utf-8", sheet_name=sheet_name).convert(buffer)
#     buffer.seek(0)
#     df = pd.read_csv(buffer)
#     return df

In [49]:
def read_parallel():
    file1 = "./data/2020-ZCOPA.xlsx"
    file2 = "./data/2021 COPA.XLSX"
    file3 = "./data/2020-21 ZCKM3.xlsx"
    file4 = "./data/Plant details COGS.xlsx"
    file5 = "./data/USL TB DATA.xlsx"
    file8 = "./data/COGS-GL and Grouping.xlsx"
    file9 = "./data/FS Tag.xlsx"
    file11 = "./data/BA Mapping.xlsx"
    paths  = [file1, file2, file3,file4,file5,file8,file9,file11]
    delayed_results = [delayed(pd.read_excel)(file_name) for file_name in paths]
    res = compute(*delayed_results, scheduler="processes")
    return tuple(res)

In [50]:
%%time
d1, d2, d3, d4, d5, d8, d9, d11 = read_parallel()

[########################################] | 100% Completed |  1min 28.2s
Wall time: 1min 28s


In [51]:
def COGS_as_per_ZCOPA(data1, data2, data4):

    plant_bp_map = dict(zip(data4["Plnt"], data4["Business Partner"]))
    plant_bp_map.update({5000: 5000})
    data1["Company"] = data1["Company Code"].map(plant_bp_map)
    data1["Company"] = data1["Company"].astype(str)
    data1["Record Type"] = data1["Record Type"].astype(str)
    data2["Company"] = data2["Company Code"].map(plant_bp_map)
    data2["Company"] = data2["Company"].astype(str)
    data2["Record Type"] = data2["Record Type"].astype(str)
    period_sum_2020 = data1[(data1["Record Type"] != "1") & (
        data1["Company"] == "5000")]["COGS - Actual"].sum()

    period_sum_2021 = data2[(data2["Record Type"] != "1") & (
        data2["Company"] == "5000")]["COGS - Actual"].sum()
    return period_sum_2020, period_sum_2021

In [52]:
COGS_as_per_ZCOPA(d1,d2,d4)

(16290501702.429998, 16982128637.73)

In [1]:
import pandas as pd

In [3]:
dl = pd.read_html("ZMM043FT.HTM")

In [9]:
header = dl[0].iloc[0]
dl[0].columns = header
df = dl[0][1:]

In [11]:
df.to_csv("ZMM043FT.csv",index=False)